In [1]:
import os
import re
import shutil
from pathlib import Path

DEPENDENCIES_SAVE_PATH = Path('dependencies')
SOURCE_CODE_SAVE_PATH = Path('source-code')
SOURCE_CODE_PATH = Path('../')
#pip install -r C://Users//Tolga//Desktop//EPOCH-IV//q3-harmful-brain-activity//submission//dependencies//requirements.txt --no-index --find-links=file://C://Users//Tolga//Desktop//EPOCH-IV//q3-harmful-brain-activity//submission//dependencies//tmp
UPDATE_DEPENDENCIES = False
UPDATE_SOURCE_CODE = True

# Manages Depenedencies

### Download Dependencies and ZIP them

In [2]:
if UPDATE_DEPENDENCIES:
    if os.path.exists(DEPENDENCIES_SAVE_PATH):
        print('Cleaning the dependencies folder')
        for filename in os.listdir(DEPENDENCIES_SAVE_PATH):
            file_path = os.path.join(DEPENDENCIES_SAVE_PATH, filename)
            if filename != 'tmp':
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
    else:
        os.makedirs(DEPENDENCIES_SAVE_PATH)

    print('Copying the requirements.txt file and excluding -e')
    with open(SOURCE_CODE_PATH / 'requirements.txt', 'r') as f:
        lines = f.readlines()
    with open(DEPENDENCIES_SAVE_PATH / 'requirements.txt', 'w') as f:
        for line in lines:
            if line.startswith('-e'):
                continue
            if line.startswith('kaggle'):
                continue
            f.write(line)

    if not os.path.exists(DEPENDENCIES_SAVE_PATH / 'tmp'):
        os.makedirs(DEPENDENCIES_SAVE_PATH / 'tmp')
    print('Downloading the dependencies')
    if not os.path.exists(DEPENDENCIES_SAVE_PATH / 'tmp'):
        os.makedirs(DEPENDENCIES_SAVE_PATH / 'tmp')
    !pip download -r {DEPENDENCIES_SAVE_PATH / 'requirements.txt'} -d {DEPENDENCIES_SAVE_PATH / 'tmp'}

    print('Zipping the downloaded dependencies')
    shutil.make_archive(DEPENDENCIES_SAVE_PATH / 'dependencies', 'zip', DEPENDENCIES_SAVE_PATH / 'tmp')
    shutil.move(DEPENDENCIES_SAVE_PATH / 'dependencies.zip', DEPENDENCIES_SAVE_PATH / 'dependencies.no_unzip')
    shutil.rmtree(DEPENDENCIES_SAVE_PATH / 'tmp')

    print('Copying the dataset-metadata.json file')
    shutil.copy('dataset-metadata-dependencies.json', DEPENDENCIES_SAVE_PATH / 'dataset-metadata.json')

    print('Excluding --find-files in requirements.txt')
    with open(DEPENDENCIES_SAVE_PATH / 'requirements.txt', 'r') as f:
        lines = f.readlines()
    with open(DEPENDENCIES_SAVE_PATH / 'requirements.txt', 'w') as f:
        for line in lines:
            if line.startswith('--find-links'):
                continue
            f.write(line)

    print('Done')

### Upload Dependencies to Kaggle as a Dataset

In [3]:
if UPDATE_DEPENDENCIES:
    # !kaggle datasets create -p ./dependencies
    !kaggle datasets version -p ./dependencies -m "Update Dependencies"


# Manage Source Code

### Copy Source Code and ZIP it

In [4]:
if UPDATE_SOURCE_CODE:
    if os.path.exists(SOURCE_CODE_SAVE_PATH):
        shutil.rmtree(SOURCE_CODE_SAVE_PATH)
    os.mkdir(SOURCE_CODE_SAVE_PATH)

    # Copy Source Code to submission/source_code
    relevant_files = ['src/', 'conf/', 'tm/', 'submit.py']
    exluded_files = ['__pycache__']
    for file in relevant_files:
        if os.path.isdir(SOURCE_CODE_PATH / file):
            # Copy directory, skip excluded files with shutil
            shutil.copytree(SOURCE_CODE_PATH / file, SOURCE_CODE_SAVE_PATH / "tmp" / file, ignore=shutil.ignore_patterns(*exluded_files))
        else:
            # Copy file
            shutil.copy(SOURCE_CODE_PATH / file, SOURCE_CODE_SAVE_PATH / "tmp" / file)

    # Zip source_code
    shutil.make_archive(SOURCE_CODE_SAVE_PATH / 'source-code', 'zip', SOURCE_CODE_SAVE_PATH / "tmp")
    shutil.rmtree(SOURCE_CODE_SAVE_PATH / "tmp")

    # Copy dataset-metadata.json to submission
    shutil.copy('dataset-metadata-source-code.json', SOURCE_CODE_SAVE_PATH / 'dataset-metadata.json')

    print('Submission files saved to source_code')

Submission files saved to source_code


### Upload Source Code

In [5]:
if UPDATE_SOURCE_CODE:
    # !kaggle datasets create -p ./source-code
    !kaggle datasets version -p ./source-code -m "Update Source Code"

Starting upload for file source-code.zip
100%|██████████████████████████████████████| 1.89G/1.89G [00:47<00:00, 43.1MB/s]
Upload successful: source-code.zip (2GB)
Dataset version is being created. Please check progress at https://www.kaggle.com/justanotherariel/epoch-hms-source-code
